# Neural Network Example

Build a 2-hidden layers fully connected neural network (a.k.a multilayer perceptron) with TensorFlow.

- Project: https://github.com/aymericdamien/TensorFlow-Examples/

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

Ok but what is a picture?


<img src="https://www.tensorflow.org/versions/r1.1/images/MNIST-Matrix.png" alt="nn" style="width: 400px;"/>


More info: http://yann.lecun.com/exdb/mnist/


## Neural Network Overview

Network structure

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>


Weights and biases detail from the hidden layer 2 to the output

<img src="https://www.tensorflow.org/versions/r1.1/images/softmax-regression-scalargraph.png" alt="nn" style="width: 400px;"/>


If we write the equations

<img src="https://www.tensorflow.org/versions/r1.1/images/softmax-regression-vectorequation.png" alt="nn" style="width: 400px;"/>

or in a compact way: <math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>y</mi>
  <mo>=</mo>
  <mtext>softmax</mtext>
  <mo stretchy="false">(</mo>
  <mi>W</mi>
  <mi>x</mi>
  <mo>+</mo>
  <mi>b</mi>
  <mo stretchy="false">)</mo>
</math>

What is softmax?
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/e348290cf48ddbb6e9a6ef4e39363568b67c09d3" alt="nn" style="width: 400px;"/>



In [4]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf
import math

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


TensorFlow deals with the so-called computational graphs which are an efficient way to execute computations in python.
A TensorFlow graph consists of:
-Placeholder variables to change the input of the graph.
-Model variables that we want to optimize.
-The model which is a mathematical function that calculates some output given the input in the placeholder variables and the model variables.
-A cost measure used to guide the optimization of the variables.
-An optimization method which updates the variables of the model.

In [57]:
# Parameters
learning_rate = 0.001
batch_size = 200
display_step = 100

# Network Parameters
n_hidden_1 = 100 # 1st layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf input
#We define a placeholder variable that serves as input
#None couse the dimension can be of any size, we specify it in the batch size
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [58]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1],mean=0.0,stddev=1.0/math.sqrt(num_input*1.0))),
    'out': tf.Variable(tf.random_normal([n_hidden_1, num_classes],mean=0.0,stddev=1.0/math.sqrt(n_hidden_1*1.0)))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [59]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.matmul(x, weights['h1']) + biases['b1']
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [60]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer

#We want to minimize the cross-entropy changing the weights and biases of the mode
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
#get a vector of booleans if predicted class == the true class of each image.
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
#cast the vector of bool to floats, so False->0 and True->1, then computes the average
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [61]:
num_steps = 1000
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

Step 1, Minibatch Loss= 3.1118, Training Accuracy= 0.155
Step 100, Minibatch Loss= 2.4294, Training Accuracy= 0.125
Step 200, Minibatch Loss= 2.1013, Training Accuracy= 0.325
Step 300, Minibatch Loss= 1.9829, Training Accuracy= 0.445
Step 400, Minibatch Loss= 1.8853, Training Accuracy= 0.530
Step 500, Minibatch Loss= 1.8326, Training Accuracy= 0.550
Step 600, Minibatch Loss= 1.7395, Training Accuracy= 0.575
Step 700, Minibatch Loss= 1.6249, Training Accuracy= 0.670
Step 800, Minibatch Loss= 1.5983, Training Accuracy= 0.640
Step 900, Minibatch Loss= 1.5221, Training Accuracy= 0.675
Step 1000, Minibatch Loss= 1.4457, Training Accuracy= 0.690
Optimization Finished!
Testing Accuracy: 0.7069


In [43]:
# Create model with 2 hidden layers and start a new train

n_hidden_1 = 256 # 1st layer number of neurons


# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1],mean=0.0,stddev=1.0/math.sqrt(num_input*1.0))),
    
    'out': tf.Variable(tf.random_normal([n_hidden_1, num_classes],mean=0.0,stddev=1.0/math.sqrt(n_hidden_1*1.0)))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    
    'out': tf.Variable(tf.random_normal([num_classes]))
}


def neural_net_2h(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.matmul(x, weights['h1']) + biases['b1']
    
    
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

logits = neural_net_2h(X)

num_steps = 1000
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

In [44]:
#modify your model introducing ReLU activation functions. try to apply an exponential decay
#to the learning rate (https://www.tensorflow.org/api_docs/python/tf/train/exponential_decay)


In [45]:
#Computes dropout (https://www.tensorflow.org/api_docs/python/tf/nn/dropout)